# Gyro Sensor Example

In [ ]:
%load_ext autoreload
%autoreload 2

from alea.sim.epa.disturbance_model import DisturbanceModel
from alea.sim.math_lib import Quaternion

import logging

import alea.sim
import numpy as np
import scipy as sp
import skyfield

from datetime import datetime
import time
from alea.sim.epa.earth_magnetic_field import EarthMagneticFieldModel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.kernel.kernel import AleasimKernel
from alea.sim.spacecraft.spacecraft import Spacecraft
import matplotlib.pyplot as plt
import logging
import cProfile
import pstats
from pstats import SortKey
from matplotlib import pyplot as plt
import skyfield.sgp4lib

from typing import Dict

import numpy as np


from alea.sim.kernel.frames import ReferenceFrame, FrameTransformation
from alea.sim.spacecraft.actuators.simple_actuators import SimpleActuator, SimpleMagnetorquer
from alea.sim.spacecraft.sensors import SimpleMagSensor, SimpleSunSensor, GyroSensor

from alea.sim.spacecraft.eps.solar_panel import SolarPanelModel
from alea.sim.spacecraft.eps.power_system import PowerSystemModel
from alea.sim.spacecraft.eps.eps import EPSModel

logging.basicConfig(level=logging.INFO)

In [ ]:
%load_ext autoreload
%autoreload 2

sim_dt=1e-3 #s
control_dt=0.1 #s
duration=100.0 #s
st=0.284e-3 #Nm
sm=0 #Am^2


control_sample_rate = int(1/control_dt)
kernel = AleasimKernel(dt=sim_dt, date=2024.2)

adyn = AttitudeDynamicsModel(kernel)
odyn = OrbitDynamicsModel(kernel)

magm = EarthMagneticFieldModel(kernel)

rwx = SimpleActuator('rw_x', kernel, st)
rwy = SimpleActuator('rw_y', kernel, st)
rwz = SimpleActuator('rw_z', kernel, st)

mtqx = SimpleMagnetorquer('mtq_x',kernel, sm)
mtqy = SimpleMagnetorquer('mtq_y',kernel, sm)
mtqz = SimpleMagnetorquer('mtq_z',kernel, sm)

sc = Spacecraft(kernel, ctrl_sample_period=control_dt)

power_sys = PowerSystemModel(kernel)

solar_panels: list[SolarPanelModel] = power_sys.solar_panels
eps = power_sys.eps
eps._force_state_of_charge(50)

DModel = DisturbanceModel(kernel)

mag_sens = SimpleMagSensor('mag_sens', kernel, sample_rate=control_sample_rate)
sun_sens = SimpleSunSensor('sun_sens', kernel, sample_rate=control_sample_rate)
gyro = GyroSensor('gyro', kernel, sample_rate=control_sample_rate)
kernel.add_model(rwx)
kernel.add_model(rwy)
kernel.add_model(rwz)
kernel.add_model(mtqx)
kernel.add_model(mtqy)
kernel.add_model(mtqz)
kernel.add_model(sc)
kernel.add_model(magm)
kernel.add_model(odyn)
kernel.add_model(adyn)
kernel.add_model(DModel)
kernel.add_model(mag_sens)
kernel.add_model(sun_sens)
kernel.add_model(gyro)
kernel.add_model(power_sys)

adyn.set_state(np.array([0.1,0.756,0.1,0,0,0,0.0]))
sc._use_quest = True

kernel.advance(duration)

kernel.kill()

## Rate Random Walk

Rate random walk is defined as a bias drift caused by accumulating white noise. It is an error source of the gyro that is time-dependent. That is, it builds up over time. Below we graph out the results from the simulation done with a "long time".

In [ ]:
objs = plt.plot(gyro.time_array, gyro.state_array[:, 8:11])
plt.title("RRW build-up")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[7:10]))
plt.ylabel('rad/s')
plt.xlabel('Time (s)')
plt.show()


## Angular Random Walk

This is the bias noise from angular random walk, which is a value given in the data sheet. It is related to the square root of the sampling rate.

In [ ]:
objs = plt.plot(gyro.time_array, gyro.state_array[:, 11:14])
plt.title("ARW values")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[7:10]))
plt.ylabel('rad/s')
plt.xlabel('Time (s)')
plt.show()